In [7]:
from bs4 import BeautifulSoup
import urllib

url = 'https://en.wikipedia.org/wiki/Data_science'
import ssl

import pandas as pd



In [8]:
with urllib.request.urlopen(url, context=ssl.SSLContext(), timeout=5) as response:
    soup = BeautifulSoup(response.read())
    

In [9]:

all_a_tags_with_href = set(filter(lambda x: 'href' in x.attrs, soup.body.find_all('a')))

links_in_page = set(map(lambda x: x.attrs['href'], all_a_tags_with_href))

links_in_page = set(map(lambda x: urllib.parse.urljoin(url, x), links_in_page))
len(links_in_page)

links_in_wikipedia = set(filter(lambda x: urllib.parse.urlparse(x).netloc.endswith('wikipedia.org'), links_in_page))
len(links_in_wikipedia)



387

In [10]:
webpages_on_wikipedia = set()

for wiki_url in links_in_wikipedia:
    try:
        with urllib.request.urlopen(wiki_url, context=ssl.SSLContext(), timeout=5) as response:
            content_type = response.info()['Content-Type'] 
            if 'text/html' in content_type:
                webpages_on_wikipedia.add(wiki_url)
    except:
        print(f'Unable to open {wiki_url}')

In [11]:
col_names = ['source_url', 'link_url', 'link_title']
df = pd.DataFrame(columns = col_names)

links_in_wikipedia = webpages_on_wikipedia

i = -1

df_dict = {}

while len(df)<100 and i<100:

    i = i+1

    j = 0
    
    url = list(links_in_wikipedia)[i]

    with urllib.request.urlopen(url, context=ssl.SSLContext(), timeout=5) as response:
        soup = BeautifulSoup(response.read())

    all_a_tags_with_href = set(filter(lambda x: 'href' in x.attrs, soup.body.find_all('a')))

    links_in_page = set(map(lambda x: x.attrs['href'], all_a_tags_with_href))

    links_in_page = set(map(lambda x: urllib.parse.urljoin(url, x), links_in_page))

    links_in_link = set(filter(lambda x: urllib.parse.urlparse(x).netloc.endswith('wikipedia.org'), links_in_page))


    if len(links_in_link) >9:
        
        while j<10:
    
            wiki_url = list(links_in_link)[j]

            try:
                with urllib.request.urlopen(wiki_url, context=ssl.SSLContext(), timeout=5) as response:
                    content_type = response.info()['Content-Type'] 
                    if 'text/html' in content_type:
                        soup = BeautifulSoup(response.read())
            except:
                print(f'Unable to open {wiki_url}')

            df_dict = {'source_url': list(links_in_wikipedia)[i], 'link_url': list(links_in_link)[j] , 'link_title':soup.title.text}
            df = df.append(df_dict, ignore_index=True)
            j = j+1









Unable to open https://my.wikipedia.org/wiki/%E1%80%A1%E1%80%91%E1%80%B0%E1%80%B8:%E1%80%98%E1%80%9A%E1%80%BA%E1%80%80%E1%80%9C%E1%80%84%E1%80%B7%E1%80%BA%E1%80%91%E1%80%AC%E1%80%B8%E1%80%9C%E1%80%B2/%E1%80%A1%E1%80%81%E1%80%BB%E1%80%80%E1%80%BA%E1%80%A1%E1%80%9C%E1%80%80%E1%80%BA%E1%80%9E%E1%80%AD%E1%80%95%E1%80%B9%E1%80%95%E1%80%B6%E1%80%95%E1%80%8A%E1%80%AC
Unable to open https://fa.wikipedia.org/wiki/ویکی‌پدیا:Text_of_Creative_Commons_Attribution-ShareAlike_3.0_Unported_License


In [6]:
df.to_csv('webcrawler.csv', sep = ',', index = False)